In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/Fake.csv')
df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
df.shape

(23481, 4)

In [6]:
true_df = pd.read_csv('/content/True.csv')
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
true_df.shape

(21417, 4)

In [8]:
#Removing the unwanted columns from both dataframes
df.drop(columns=['title', 'subject', 'date'], inplace=True)
true_df.drop(columns=['title', 'subject', 'date'], inplace=True)
df.head()

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...


In [9]:
true_df.head()

,text
0,WASHINGTON (Reuters) - The head of a conservat...
1,WASHINGTON (Reuters) - Transgender people will...
2,WASHINGTON (Reuters) - The special counsel inv...
3,WASHINGTON (Reuters) - Trump campaign adviser ...
4,SEATTLE/WASHINGTON (Reuters) - President Donal...


In [10]:
df.isna().sum()

,0
text,0


In [12]:
true_df.isna().sum()

,0
text,0


In [13]:
#Assiging the labels, because they data didn't have the label column
df["label"] = 1
true_df["label"] = 0

In [14]:
df.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


In [15]:
fake = df.iloc[:5000]
true = true_df.iloc[:5000]

In [16]:
fake.shape, true.shape

((5000, 2), (5000, 2))

In [17]:
df = pd.concat([fake, true], axis = 0)

In [18]:
df.shape

(10000, 2)

In [19]:
#Text Preprocessing
import spacy

nlp = spacy.load('en_core_web_sm')
def preprocessing(text):
  doc = nlp(text.lower())
  processed = [
      word.lemma_ for word in doc if not word.is_stop and not word.is_punct
  ]
  return " ".join(processed)

In [20]:
df['final'] = df['text'].apply(preprocessing)

In [21]:
df.head()

,text,label,final
0,Donald Trump just couldn t wish all Americans ...,1,donald trump couldn t wish americans happy new...
1,House Intelligence Committee Chairman Devin Nu...,1,house intelligence committee chairman devin nu...
2,"On Friday, it was revealed that former Milwauk...",1,friday reveal milwaukee sheriff david clarke c...
3,"On Christmas day, Donald Trump announced that ...",1,christmas day donald trump announce work f...
4,Pope Francis used his annual Christmas Day mes...,1,pope francis annual christmas day message rebu...


In [22]:
#dividing the dataframe
X = df['final']
y = df['label']

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [25]:
type(X_train), type(y_train)

(pandas.core.series.Series, pandas.core.series.Series)

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

classifier = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('Random Forest', RandomForestClassifier(n_jobs = -1, random_state= 42))
])

classifier.fit(X_train, y_train)

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Random Forest',
                 RandomForestClassifier(n_jobs=-1, random_state=42))])

In [27]:
classifier.score(X_test,y_test)

0.9995

In [28]:
pred = classifier.predict(X_test)

In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print(f"Accuracy Score: {accuracy_score(y_test, pred)}")
print(f"Classification report: {classification_report(y_test, pred)}")
print(f"Confustion Matrix: {confusion_matrix(y_test, pred)}")

Accuracy Score: 0.9995
Classification report:               precision    recall  f1-score   support

           0       1.00      1.00      1.00       988
           1       1.00      1.00      1.00      1012

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

Confustion Matrix: [[ 987    1]
 [   0 1012]]


In [30]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Build pipeline with XGBoost
classifier = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range=(1,2), max_features=10000)),
    ('xgb', XGBClassifier(
        n_estimators=300,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        use_label_encoder=False,
        random_state=42
    ))
])

# Fit
classifier.fit(X_train, y_train)

# Predict
y_pred = classifier.predict(X_test)

# Evaluate
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("Classification report:", classification_report(y_test, y_pred))
print("Confusion Matrix:", confusion_matrix(y_test, y_pred))


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [14:20:52] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy Score: 1.0
Classification report:               precision    recall  f1-score   support

           0       1.00      1.00      1.00       988
           1       1.00      1.00      1.00      1012

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

Confusion Matrix: [[ 988    0]
 [   0 1012]]


In [31]:
classifier

Pipeline(steps=[('vectorizer_tfidf',
                 TfidfVectorizer(max_features=10000, ngram_range=(1, 2))),
                ('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.8, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric='logloss',
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None, ...))])